<a href="https://colab.research.google.com/github/hanna-joo/project/blob/master/Consumption_Trends_2020/00_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive/Colab Notebooks/04_kdx/data_in'

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/04_kdx/data_in


In [3]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm

## 1. 온라인 구매이력 데이터 전처리


### 1.1. 데이터 파일 합치기
- 64개의 데이터 파일을 컬럼명 기준으로 합치겠다
- 데이터 출처: Mcorporation
- 데이터 형태: 업종별 구매이력 데이터 엑셀파일 총 64개

In [75]:
# 소비업종 카테고리 가져오기
categories = pd.read_excel('/content/gdrive/My Drive/Colab Notebooks/04_kdx/data_in/Mcorporation/KDX_CONTEST_파일정의서.xlsx')['xlsx 파일 이름'].values
categories = list(categories)
categories[:5]

['PC사무기기', 'TV홈시어터', '가공식품', '가방지갑잡화', '건강식품']

In [77]:
mcorp_all = pd.DataFrame()
for i, category in enumerate(categories):
  path = '/content/gdrive/My Drive/Colab Notebooks/04_kdx/data_in/Mcorporation/{}.xlsx'.format(category)
  df = pd.read_excel(path)
  df['카테고리명'] = category
  mcorp_all = pd.concat((mcorp_all, df), axis=0)
  print(category, i)

print('*'*50)

# 데이터 제대로 나왔는지 확인
if len(mcorp_all.카테고리명.unique())==64:
  print('데이터 병합 이상없음')
else:
  print('데이터 병합 에러')

PC사무기기 0
TV홈시어터 1
가공식품 2
가방지갑잡화 3
건강식품 4
골프용품 5
공구류 6
구기 7
기타 스포츠 8
낚시 9
남성의류 10
계절가전 11
노트북 12
농축수산물 13
도서음반 14
등산용품 15
메이크업 16
문구사무용품 17
미용가전 18
반려동물 19
생활용품 20
생활가구 21
생활가전 22
서비스티켓 23
성인용품 24
세탁청소세면 25
수납용품 26
수납가구 27
수영 28
스키보드 29
스킨케어 30
스포츠의류 31
신발 32
악세서리시계주얼리 33
안전용품 34
언더웨어 35
업소위생용품 36
여성의류 37
국내외여행 38
완구키덜트게임 39
욕실용품 40
욕실가전 41
유아용품 42
유아패션 43
음료 44
음향가전 45
인테리어용품 46
자동차용품 47
자전거사이클보드인라인 48
주방가전 49
주방수납잡화 50
주방식기용기 51
주방조리기구 52
출산임부용품 53
취미악기 54
침실가구 55
침실인테리어 56
카메라캠코더 57
캠핑용품 58
테마의류 59
헤어바디용품 60
헬스기구용품 61
건강관련용품 62
휴대폰악세서리 63
**************************************************
데이터 병합 이상없음


In [78]:
# 전체 원본 데이터 저장하기 (파일명: mcorp_all.csv)
mcorp_all.to_csv('/content/gdrive/Shared drives/청년취업아카데미_KDX한국데이터거래소/Data/mcorp_all.csv', index=False)

### 1.2. 데이터 정제하기
- 1.2.1. 고객 연령대 데이터 수정 (고객나이)
  + 0보다 작은 값과 0인 값에 해당하는 행 삭제(데이터 담당자 확인 결과 의미 없는 값으로 판명)
  + 80, 90, 100, 110, 120 인 값은 모두 70대로 변경
  + 연령대 70은 70대 이상으로 간주
- 1.2.2. 데이터 기간 설정 (구매날짜)
  + 구매 날짜가2020년 6월까지만 데이터 존재
  + 2019년, 2020년 데이터 모두 1-6월까지만 사용
- 1.2.3. 컬럼명 통합하기
  + 편의를 위해 컬럼명 변경


  |_변경전_||구매날짜|카테고리명|고객성별|고객나이|OS유형|구매금액|구매수|
  |---||---|---|---|---|---|---|---|
  |_변경후_||**소비일자**|**소비업종**|**성별**|**연령대**|**OS유형**|**소비금액**|**소비건수**|

In [79]:
mcorp_all = pd.read_csv('/content/gdrive/Shared drives/청년취업아카데미_KDX한국데이터거래소/Data/Mcorp_all.csv', encoding='utf8')

# 고객 연령대 값 확인
mcorp_all.고객나이.unique()

array([ 10,  20,  30,  40,  50,  60,  70,  80,  90,   0, -30, -50, -60,
       -40, 100, -20, -80, -70, 110, -10, 120])

In [81]:
mcorp = mcorp_all.copy()

# 1.2.1. 연령대 수정하기
temp = mcorp[~mcorp['고객나이'].isin([0,-10,-20,-30,-40,-50,-60,-70])]
temp['고객나이'] = [70 if x>=70 else x for x in temp['고객나이']]
mcorp = temp

# 1.2.2. 데이터 기간 설정
temp19 = mcorp[mcorp.구매날짜<=20190630]
temp20 = mcorp[(mcorp.구매날짜>=20200101) & (mcorp.구매날짜<=20200630)]
temp1920 = mcorp[(mcorp.구매날짜<=20190630)|(mcorp.구매날짜>=20200101) & (mcorp.구매날짜<=20200630)]

if (len(temp19.구매날짜) + len(temp20.구매날짜) == len(temp1920.구매날짜)):
  print('1월-6월까지의 데이터가 정상 추출되었습니다')
  print('*'*50)

mcorp = temp1920

# 1.2.3. 컬럼명 변경하기
mcorp.columns = ['소비일자','소비업종','성별','연령대','OS유형','소비금액','소비건수']

1월-6월까지의 데이터가 정상 추출되었습니다
**************************************************


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [82]:
mcorp.head()

,소비일자,소비업종,성별,연령대,OS유형,소비금액,소비건수
0,20190101,PC사무기기,F,10,IOS,352443,13
1,20190101,PC사무기기,F,10,WINDOWS,84000,1
2,20190101,PC사무기기,F,10,안드로이드,80870,8
3,20190101,PC사무기기,F,10,없음,3700,1
4,20190101,PC사무기기,F,20,IOS,27714776,381


In [83]:
# 최종 분석용 데이터 저장하기 (파일명: mcorp_edit.csv)
mcorp.to_csv('/content/gdrive/Shared drives/청년취업아카데미_KDX한국데이터거래소/Data/mcorp_edit.csv', index=False)